## Check for structure and format errors

Kingfisher Process checks data against the OCDS schema using [CoVE](https://github.com/OpenDataServices/cove). For release collections, Kingfisher Process stores check results in the `release_check` table. For record collections, Kingfisher Process stores check results in the `record_check` table.

### Error summary

Generate a summary of errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_error_summary <<

SELECT
	collection_id,
	'release' AS collection_type,
	errors ->> 'type' AS error_type,
	LEFT(errors ->> 'description', 49000) AS error,
	COUNT(*) AS count
FROM
	release_check rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
JOIN RELEASE r ON
	rc.release_id = r.id
WHERE
	collection_id IN :collection_ids
GROUP BY
	collection_id,
	error_type,
	error
UNION ALL
SELECT
	collection_id,
	'record' AS collection_type,
	errors ->> 'type' AS error_type,
	LEFT(errors ->> 'description', 49000) AS error,
	COUNT(*) AS count
FROM
	record_check rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
JOIN record r ON
	rc.record_id = r.id
WHERE
	collection_id IN :collection_ids
GROUP BY
	collection_id,
	error_type,
	error
ORDER BY
	collection_id,
	count DESC

In [ ]:
structure_and_format_error_summary

### Error details

Generate a full list of errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_errors <<

SELECT
	collection_id,
	'release' AS collection_type,
	errors ->> 'type' AS error_type,
	errors ->> 'field' AS field,
	ocid,
	LEFT(errors ->> 'description', 49000) AS error,
	COUNT(*) AS count
FROM
	release_check rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
JOIN RELEASE r ON
	rc.release_id = r.id
WHERE
	collection_id IN :collection_ids
GROUP BY
	collection_id,
	error_type,
	field,
	error,
	ocid
UNION ALL
SELECT
	collection_id,
	'record' AS collection_type,
	errors ->> 'type' AS error_type,
	errors ->> 'field' AS field,
	ocid,
	LEFT(errors ->> 'description', 49000) AS error,
	COUNT(*) AS count
FROM
	record_check rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
JOIN record r ON
	rc.record_id = r.id
WHERE
	collection_id IN :collection_ids
GROUP BY
	collection_id,
	error_type,
	field,
	error,
	ocid
ORDER BY
	count DESC

In [ ]:
structure_and_format_errors